In [3]:
import src.data.utilities as utils
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd

In [5]:
## DATASETS ##
# 1. Sentencing data 2017–21 (from: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1076592/Data-behind-interactive-tools-3.zip)
cols = ['Police Force Area', 'Year', 'Sex', 'Age group', 'Offence group', 'Sentence Outcome', 'Custodial Sentence Length','Sentenced']
df = utils.loadData("data/external/sentencing.csv", cols=cols)
df.head()

,Year,Offence group,Sex,Age group,Police Force Area,Sentence Outcome,Custodial Sentence Length,Sentenced
0,2017,01: Violence against the person,02: Male,02: Young adults,Greater Manchester,07: Total Immediate custody,23: Custody - Life,1
1,2017,01: Violence against the person,02: Male,03: Adults,West Yorkshire,07: Total Immediate custody,23: Custody - Life,1
2,2017,01: Violence against the person,02: Male,03: Adults,Metropolitan Police,07: Total Immediate custody,23: Custody - Life,1
3,2017,01: Violence against the person,02: Male,03: Adults,West Yorkshire,07: Total Immediate custody,23: Custody - Life,2
4,2017,01: Violence against the person,02: Male,01: Children,Metropolitan Police,07: Total Immediate custody,23: Custody - Life,2


In [6]:
# 2. Court outcomes by police force area 2009–2019 (from: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/888561/csvs-behind-data-tools-2-2019.zip)
cols_2009 =['Police Force Area', 'Year of Appearance', 'Sex', 'Age Group', 'Offence Group', 'Outcome', 'Custodial Sentence Length','Count'] 
df_2009 = utils.loadData('data/external/court-outcomes-by-PFA-2019.csv', cols=cols_2009)
df_2009.head()

,Police Force Area,Year of Appearance,Sex,Age Group,Offence Group,Outcome,Custodial Sentence Length,Count
0,Avon and Somerset,2009,01: Male,01: Children,04: Theft Offences,13: Community sentence,NaN,1
1,Avon and Somerset,2009,01: Male,01: Children,03: Robbery,13: Community sentence,NaN,1
2,Avon and Somerset,2009,01: Male,01: Children,11: Summary non-motoring,13: Community sentence,NaN,1
3,Avon and Somerset,2009,01: Male,01: Children,11: Summary non-motoring,13: Community sentence,NaN,1
4,Avon and Somerset,2009,01: Male,01: Children,01: Violence against the person,08: Committed for trial,NaN,1


In [7]:
#Dropping duplicate data from 2009 dataset that also appears in df
filt = df_2009['Year of Appearance'] < 2017
df_2009 = df_2009[filt].copy()

In [8]:
df_list = utils.dataframeList(locals())

In [9]:
#Standardising variable names
for data in df_list:
    utils.lcColumns(data)
    utils.renameColumns(data, columns={
        'year_of_appearance': 'year',
        'offence_group': 'offence',
        'police_force_area': 'pfa',
        'sentence_outcome': 'outcome',
        'custodial_sentence_length': 'sentence_length',
        'sentenced': 'freq',
        'count': 'freq'}
        )
    # utils.orderColumns(data, column_order = ['year', 'pfa', 'sex', 'age_group', 'offence', 'outcome', 'sentence_length', 'freq'])

In [26]:

def tidy_elements(data):
    regex = {r"^\S*: \S* - ": "",
             r"\d\d: ": "",
            "Total ": "",
            "(Over)": "More than",
            "( and including)": "",
            "(to less than)": "and under",
            "Life$": "Life sentence"
             }
    return data.map(regex)

In [29]:
df_combined = pd.concat(df_list)
tidy_elements(df_combined)

AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
df_combined = pd.concat(df_list)
utils.tidy_elements(df_combined)

In [23]:
df_combined.dtypes

year                int64
offence            object
sex                object
age_group          object
pfa                object
outcome            object
sentence_length    object
freq                int64
dtype: object

In [21]:
convert_dict = {'outcome': "category",
                'sentence_length': "category"
                }

In [24]:
df_combined = df_combined.astype(convert_dict)

In [25]:
df_combined.dtypes

year                  int64
offence              object
sex                  object
age_group            object
pfa                  object
outcome            category
sentence_length    category
freq                  int64
dtype: object